# Automatic identification and structuring of AI companies for product creation

### Author: Chaitali Suhas Bagwe (cbagwe@mail.uni-paderborn.de)

***
#### Install Necessary Libraries

Remove the comment char '#' for installing the libraries

In [1]:
#!pip install requests
#!pip install beautifulsoup4

***
#### Import Libraries

In [2]:
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm,metrics
from urllib.request import Request, urlopen
import yake

import pandas as pd
import re
import requests
import scrapy

***
#### Read the data from the Excel File
In this block we read the data from excel file to the pandas dataframe and change the column name of "Relevance of service/app and fit to the portfolio" to "Relevance" for better readability/extraction. Since we are only interested in product engineering companies we will mark the "1 - product engineering" as "1" and others as "0". This makes the classification problem into binary classification.

In [3]:
#Read the input Excel file
dataset = pd.read_excel('App and service store_Long List.xlsx', sheet_name='App and service store_Long List')

#Change column names
dataset.rename(columns = {'Relevance of service/app and fit to the portfolio':'Relevance'}, inplace = True)

#Change Relevance values
dataset.Relevance.replace('1 - product engineering',1, inplace = True)
dataset.Relevance.replace('2 - production',0, inplace = True)
dataset.Relevance.replace('3- AI general',0, inplace = True)
dataset.Relevance.replace('4 - cross sectional processes',0, inplace = True)
dataset.Relevance.replace('5 - not relevant',0, inplace = True)

dataset["Relevance"]

c:\users\chaitali bagwe\appdata\local\programs\python\python38\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


0      0
1      0
2      0
3      0
4      0
      ..
433    0
434    1
435    1
436    0
437    0
Name: Relevance, Length: 438, dtype: int64

***
#### Segregration of Test and Train data for our ML model
The sampling helps us in having random and not fixed input to the model. We have used 60 samples of product engineering companies and 60 samples of other companies as our training dataset. Thus the size of training data is 120. For test data we have used random 20 samples from each type of companies. Thus the size of testing data is 40.

In [4]:
# Segregrate the product engineering and other data from the dataset
prod_engg_data = dataset[dataset['Relevance'] == 1]
other_data = dataset[dataset['Relevance'] == 0]

# Generate train and test data by random sampling of product engineering and other data and then appending it together.
train_data = prod_engg_data.sample(60).append(other_data.sample(60))
test_data = prod_engg_data.sample(20).append(other_data.sample(20))

***
### Text Preprocessing

#### 1) Setting up stop-words and stemmer
Stop Words : Removing commonly used english stop words like *and*, *the*, *a*, *an*, etc. We have also removed the words that are not useful for us in the context of websites. These include the words from header and footer of the websites, dropdown menu words of the websites, terms and conditions, etc.

Stemmer : Reducing the word to its word stem; *extraction* -> *extract*

In [5]:
stemmer = PorterStemmer()

stop_words = stopwords.words("english")
# Add custom stop words (frequently occuring but add no value)
stop_words += ['about', 'us', 'contact', 'how','login', 'hello','email','home','blog','terms','conditions']

***
#### 2) Clean Text Function

This function cleans the text received by the variable "text". Cleaning includes removal of stopwords, white spaces, html tags, numbers, special characters and punctuations. This function also tokenizes the text sent to it and performs stemming on each word.

In [6]:
def clean_text(text):
    
    # remove white spaces, html tags, numbers, special characters, punctuations
    RE_WSPACE = re.compile(r"\s+", re.IGNORECASE)
    RE_TAGS = re.compile(r"<[^>]+>")
    RE_ASCII = re.compile(r"[^A-Za-zÀ-ž ]", re.IGNORECASE)
    RE_SINGLECHAR = re.compile(r"\b[A-Za-zÀ-ž]\b", re.IGNORECASE)

    text = re.sub(RE_TAGS, " ", text)
    text = re.sub(RE_ASCII, " ", text)
    text = re.sub(RE_SINGLECHAR, " ", text)
    text = re.sub(RE_WSPACE, " ", text)

    word_tokens = word_tokenize(text)
    words_tokens_lower = [word.lower() for word in word_tokens]

    # perform stemming on each word
    words_filtered = [
        stemmer.stem(word) for word in words_tokens_lower if word not in stop_words
    ]

    text_clean = " ".join(words_filtered)
    return text_clean

***
#### 3) Initialize the keyword extractor from Yake library

In [7]:
max_ngram_size = 1
deduplication_threshold = 0.9
num_of_keywords = 30
window_size = 1

kw_extractor = yake.KeywordExtractor(n=max_ngram_size, dedupLim=deduplication_threshold, top=num_of_keywords, windowsSize=window_size)

***
### Miscellaneous Functions

#### 1) Read Content from URL of the company

BeautifulSoup library is used for the extraction of content from the given URL. Additionally, the footer attributes and the website's cookies attributes are removed in this function.

In [8]:
def read_url_content(URL,page):
    # read the content
    soup = BeautifulSoup(page.content, "html.parser")
    
    # remove the website cookies content
    for div in soup.find_all('div', attrs={'data-nosnippet' : 'true'}):
        div.decompose()
        
    # remove footer
    for footer in soup.find_all('footer'):
        footer.decompose()
        
    # return the cleaned content
    return clean_text(soup.text)

***
#### 2) Parse the required contents from the dataframe.
This function reads the data from the dataframe and passes the URL of each company to the read_url_content() function. A try and catch block is added for checking if the URL is accessible or not.

In [9]:
def get_cleaned_webdata(dataframe):
    # create an empty list of page data
    page_data = []
    
    for index, row in dataframe.iterrows():
        URL = row["Link"]
        try:
            # access the URL
            page = requests.get(URL)
            # append the URL content to the list
            page_data.append(read_url_content(URL,page))
        except(ConnectionError, Exception):
            # for websites not accessible append empty string to the list
            page_data.append("")
    return page_data

***
#### 3) Metrics Function
Calculates the metrics for the given actual and predicted datasets.

In [10]:
def print_statistics(actual, predicted):
    avg = 'weighted'
    print("Accuracy:",metrics.accuracy_score(actual, predicted))
    print("Precision:",metrics.precision_score(actual, predicted, average=avg))
    print("Recall:",metrics.recall_score(actual, predicted, average=avg))
    print("F1 score:",metrics.f1_score(actual, predicted, average=avg))

***
#### 4) Extract Keywords Function

In [11]:
def extract_keywords(string):
    keywords = kw_extractor.extract_keywords(string)
    keywords = [x for (x,_) in keywords]
    return " ".join(keywords)

***
#### Get the cleaned webdata for train and test dataset
The cleaned webdata is stored in the "WebData" column created in pandas dataframe for train and test dataset respectively.

In [12]:
#Clean train and test data   
train_data["WebData"] = get_cleaned_webdata(train_data)
test_data["WebData"] = get_cleaned_webdata(test_data)

***
#### Perform keyword extraction on the cleaned webdata 

In [22]:
yake = "true"
if yake == "true":
    train_data["Keywords"] = train_data["WebData"].apply(extract_keywords)
    test_data["Keywords"] = test_data["WebData"].apply(extract_keywords)

***
#### Feature extraction using TF-IDF
TF-IDF measures how relvant a word is when compared with the entire document.

In [24]:
tfidf_vectorizor = TfidfVectorizer()

In [28]:
if yake == "true":
    # Transform the train and test cleaned input webdata to tfidf vectors respectively
    train_input = tfidf_vectorizor.fit_transform(train_data["Keywords"]).toarray().tolist()
    test_input = tfidf_vectorizor.transform(test_data["Keywords"]).toarray().tolist()
else:
    # Transform the train and test cleaned input webdata to tfidf vectors respectively
    train_input = tfidf_vectorizor.fit_transform(train_data["WebData"]).toarray().tolist()
    test_input = tfidf_vectorizor.transform(test_data['WebData']).toarray().tolist()

In [29]:
# Convert the train and test output data to list.
train_output = train_data['Relevance'].tolist()
test_output = test_data['Relevance'].tolist()

***
#### SVM Classifier

In [30]:
#Create a svm Classifier
clf = svm.SVC(kernel='linear') # Linear Kernel

#Train the model using the training sets
clf.fit(train_input, train_output)

#Predict the response for test dataset
y_pred = clf.predict(test_input)

***
#### Print the statistics for SVM Classifier

In [31]:
print_statistics(test_output,y_pred)

Accuracy: 0.9
Precision: 0.9040404040404042
Recall: 0.9
F1 score: 0.8997493734335841


In [ ]:
#soup_initial = re.sub("\s{2,}[a-z]+\s{2,}", " ", soup.text,flags=re.IGNORECASE)
#print(clean_text(soup.text))